In [1]:
import pandas as pd
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_pickle('/content/drive/My Drive/Northwestern MS/453 - Natural Language Processing/imdb/imdb_dataset.pkl')
df.head()

,review,sentiment,processed
0,One of the other reviewers has mentioned that ...,positive,"[[reviewer, mention, watch, oz, episode, hook,..."
1,A wonderful little production. <br /><br />The...,positive,"[[wonderful, little, production, filming, tech..."
2,I thought this was a wonderful way to spend ti...,positive,"[[think, wonderful, way, spend, time, hot, sum..."
3,Basically there's a family where a little boy ...,negative,"[[basically, family, little, boy, jake, think,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[[petter, matteis, love, time, money, visually..."


In [4]:
embeddings_df = pd.read_pickle('/content/drive/My Drive/Northwestern MS/453 - Natural Language Processing/imdb/embeddings_df.pkl')
embeddings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.501618,0.164232,-0.687424,0.456446,-0.894968,-2.257915,0.228125,1.964802,-0.481894,0.662188,...,2.392052,0.079842,2.045716,-1.270257,1.064509,1.723800,0.628179,-0.688859,-1.628123,-1.355341
1,-0.365358,2.384981,-0.279481,0.352401,-0.470002,0.050526,-0.866197,-0.412319,0.651297,-2.049760,...,0.767018,1.968958,-0.728388,-0.284288,0.689161,1.379422,0.503419,-1.220167,-1.444172,0.647671
2,0.946937,1.573278,-1.519988,0.782868,0.184616,-0.853950,0.537544,0.863877,-0.742539,-0.402138,...,0.788282,-1.107653,0.709821,1.976070,-0.354313,1.319767,1.775188,1.175694,-1.982500,0.457960
3,0.537245,-0.687223,-1.167942,0.429164,0.213441,-1.387397,-0.745562,-0.722211,-0.085279,-0.222465,...,2.237369,-2.202821,0.442173,1.106392,-0.997359,0.794328,-0.201841,-0.040549,1.549424,-1.393315
4,0.502567,0.749984,-1.455203,-1.174098,0.804368,-2.091628,-1.033265,-0.807077,-0.011158,-1.278914,...,0.177937,-0.942314,0.043687,-1.070671,1.851442,1.642835,-2.512048,2.696857,-0.109828,-0.837788


In [6]:
# embeddings_df = your_dataframe_of_embeddings

# Define the parameter grid
eps_values = np.arange(0.3, 2.2, 0.2)  # Broadened range for EPS
min_samples_values = (50, 100, 200, 250, 300)  # Adjusted values as per your need

# Store the best configuration
best_eps = None
best_min_samples = None
best_silhouette = -1
best_num_clusters = 0  # To store the best number of clusters

# Grid search
for eps in eps_values:
    for min_samples in min_samples_values:
        # Initialize DBSCAN with current parameters
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine')
        # Fit the model
        clusters = dbscan.fit_predict(embeddings_df)

        # Calculate the number of clusters (excluding noise)
        num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)

        # Calculate the silhouette score only if more than one cluster and noise (-1) is present
        if num_clusters > 1:
            score = silhouette_score(embeddings_df, clusters)
            print(f"EPS: {eps}, min_samples: {min_samples}, Silhouette Score: {score}, Number of Clusters: {num_clusters}")

            # Check if the current score is better and update best parameters
            if score > best_silhouette:
                best_eps = eps
                best_min_samples = min_samples
                best_silhouette = score
                best_num_clusters = num_clusters
        else:
            score = None  # No valid score since valid clusters aren't formed
            print(f"EPS: {eps}, min_samples: {min_samples}, Silhouette Score: N/A, No valid clusters formed")

print(f"Best EPS: {best_eps}, Best min_samples: {best_min_samples}, Best Silhouette Score: {best_silhouette}, Best Number of Clusters: {best_num_clusters}")


EPS: 0.3, min_samples: 50, Silhouette Score: N/A, No valid clusters formed
EPS: 0.3, min_samples: 100, Silhouette Score: N/A, No valid clusters formed
EPS: 0.3, min_samples: 200, Silhouette Score: N/A, No valid clusters formed
EPS: 0.3, min_samples: 250, Silhouette Score: N/A, No valid clusters formed
EPS: 0.3, min_samples: 300, Silhouette Score: N/A, No valid clusters formed
EPS: 0.5, min_samples: 50, Silhouette Score: N/A, No valid clusters formed
EPS: 0.5, min_samples: 100, Silhouette Score: N/A, No valid clusters formed
EPS: 0.5, min_samples: 200, Silhouette Score: N/A, No valid clusters formed
EPS: 0.5, min_samples: 250, Silhouette Score: N/A, No valid clusters formed
EPS: 0.5, min_samples: 300, Silhouette Score: N/A, No valid clusters formed
EPS: 0.7, min_samples: 50, Silhouette Score: N/A, No valid clusters formed
EPS: 0.7, min_samples: 100, Silhouette Score: N/A, No valid clusters formed
EPS: 0.7, min_samples: 200, Silhouette Score: N/A, No valid clusters formed
EPS: 0.7, min_s